In [2]:
# 1. Import packages
import nltk
import numpy as np
import pandas as pd
import os, sys

In [7]:
# 2. Import data needed
df = pd.read_csv('./emails.train.csv')
#df_pos = df[df['spam']==1]

# 3. Start to clean the data 
#text = np.random.choice(df['text'])
#path = os.getcwd()
#os.chdir(path)
#train_df = pd.read_csv('./emails.train.csv', nrows=20, delimiter=',')

from nltk.tokenize import RegexpTokenizer 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

tokenizer = RegexpTokenizer(r'\w+') #tokens without special character
stop_words = set(stopwords.words('english')) #stop words remover
ps = PorterStemmer() #stemming words
lemmatizer = WordNetLemmatizer() #lemmatizing words

del_words = ['subject', '_']

print('--dirty--')
#print(text)

#print(stop_words)

def select_vocabulary(dataframe, topN=1000):
    #for w in dataframe.str.lower():
    word2freq = dict()
    for line in dataframe:
        line = re.sub("(^|\W)\d+($|\W)", " ", line)
        word_tokens = tokenizer.tokenize(line) # tokenize all the words in line
        #words = line.split()
        for word in word_tokens: 
            if word not in stop_words:
                if word not in del_words: # we check if the current words were looking at are not in the stopwords, we wnat to use as ft
                    #if word != 'Subject':
                        word = ps.stem(word) # only for the words that I am going to use, with "word"we overwrite waht used to be the word that is stemmed
                        #word2freq[word] = word2freq.setdefault(word, 0) +1

    word_freq = [ (word,freq) for word,freq in word2freq.items() ]
    
    # sort according to freq, in descending order.
    #word_freq.sort(key=lambda x:x[1], reverse=True) 
    
    # show selection results
#     print("%-10s  %10s" % ('word', 'frequency'))
#     print("-------------------------")
#     for i in range(150):
#         print("%-10s  %10d" % (word_freq[i]))
    #print("...\n")
    #return([x[0] for x in word_freq[:topN]])

vocabulary = select_vocabulary(df['text'])
print(vocabulary)
#word2ind   = dict(zip(vocabulary, range(len(vocabulary))))
#print(word2ind)


--dirty--
None


In [114]:
def extract_Bag_of_Word_feature(dataframe):
    BoWs = np.zeros((len(dataframe), len(vocabulary)), dtype=np.float32)

    for i, line in enumerate(dataframe):
        for word in line.split():   ## these are the not stemmed words
            word = ps.stem(word)    # looked for the stemmed words in our dict cause we stemmed them 
            word_ind = word2ind.get(word, -1) #looks in dict
            if(word_ind>=0):
                BoWs[i, word_ind] += 1

        if i%1000==0:
            print("%4d / %d " % (i, len(dataframe)))
    print("%4d / %d " % (i, len(dataframe)))
            
    return BoWs


# Make sure use cleaned version
train = pd.read_csv('./emails.train.csv')
test  = pd.read_csv('./emails.test.csv')

# Get labels
Y_train = train['spam']
Y_test  = test['spam']

print("Extracting feature for train ...")
X_train = extract_Bag_of_Word_feature(train['text']) # these are our features
print(X_train)

print("Extracting feature for test ...")
X_test  = extract_Bag_of_Word_feature(test[ 'text']) # this is prediction 

print('Finish.')

Extracting feature for train ...
   0 / 4021 
1000 / 4021 
2000 / 4021 
3000 / 4021 
4000 / 4021 
4020 / 4021 
[[  0.   0.   0. ...,   0.   0.   0.]
 [  0.   0.   0. ...,   0.   0.   0.]
 [  0.   0.   0. ...,   0.   0.   0.]
 ..., 
 [  4.   4.   2. ...,   0.   0.   0.]
 [  0.  11.   7. ...,   0.   0.   0.]
 [  0.   0.   0. ...,   0.   0.   0.]]
Extracting feature for test ...
   0 / 1707 
1000 / 1707 
1706 / 1707 
Finish.


In [115]:
from sklearn.metrics import average_precision_score, accuracy_score
from sklearn.svm import SVC

model = SVC()

model.fit(X_train, Y_train) # fit is for our training data, the extracted bag of words

def eval(model, X_test, Y_test, method=''):
    print("====== Performance of: {method} =======".format(method=method))
    
    # Predict decision labels.
    Y_pred  = model.predict(X_test)  #we need this 
    print("Metric[{metric:20s}]  {score:-3f}".format( metric="Accuracy", 
                                              score=accuracy_score(Y_test, Y_pred)) )

    # Predict confidence scores.
    Y_score = model.decision_function(X_test)    
    print("Metric[{metric:20s}]  {score:-3f}".format( metric="Average Precision", 
                                              score=average_precision_score(Y_test, Y_score)) )

    # write to submit format
    outf = 'solution.%s.csv'% method
    with open( outf, 'w') as f:
        f.write('id,spam\n')
        for i in range(len(Y_pred)):
            # print test['id'][0]
            f.write('%s,%s\n' % (test['id'][i], Y_pred[i]) )
    print("[output] "+outf)
    
    
# evaluate current model
eval(model, X_test, Y_test, method='SVM')

====== Performance of: SVM =======
Metric[Accuracy            ]  0.967194
Metric[Average Precision   ]  0.983188
[output] solution.SVM.csv
